In [6]:
# Import modules
import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import numpy as np
import seaborn as sn
import csv
import matplotlib.pyplot as plt

I am using Jumia's link to get customer reviews and comments about Starlink . [Click here](https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/) to get the link

In [15]:
# To extract reviews from multiple pages
all_reviews = []
for page_num in range(1, 10):
    # Format and print the URL to check correctness
    url_source = 'https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page={page_num}'
    url = url_source.format(page_num=page_num)
    print(f"Fetching page: {url}")
    
    # Send the request and check status
    r = requests.get(url)
    if r.status_code != 200:
        print(f"Failed to fetch page {page_num}, status code: {r.status_code}")
        continue

    # Extract data using BeautifulSoup
    soup = BeautifulSoup(r.content, 'lxml')

    # Extract the reviews
    reviews = soup.find_all('p', {'class': "-pvs"})
    div_reviews = soup.find_all('div', {'class': "-m -fs16 -pvs"})

    # Check if reviews were found
    if not reviews and not div_reviews:
        print(f"No reviews found on page {page_num}.")
    else:
        print(f"Found {len(reviews) + len(div_reviews)} reviews on page {page_num}.")

    # Add each review to all_reviews
    for review in reviews:
        all_reviews.append(review.text.strip())
    
    for div_review in div_reviews:
        all_reviews.append(div_review.text.strip())

Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=1
Found 10 reviews on page 1.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=2
Found 10 reviews on page 2.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=3
Found 10 reviews on page 3.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=4
Found 10 reviews on page 4.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=5
Found 10 reviews on page 5.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=6
Found 10 reviews on page 6.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?page=7
Found 10 reviews on page 7.
Fetching page: https://www.jumia.co.ke/catalog/productratingsreviews/sku/BR269EA1PRKULNAFAMZ/?pag

In [16]:
# Save reviews to CSV
if all_reviews:
    with open('jumia_reviews.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Review'])  # Header row
        for review in all_reviews:
            writer.writerow([review])  # Write each review on a new row
    print("Reviews saved to 'jumia_reviews.csv'")
else:
    print("No reviews to save.")

Reviews saved to 'jumia_reviews.csv'
